Generate a dcb master dataframe (with Daily dcb and/or Monthly code P1P2 and P1C1)

In [ ]:
from pathlib import Path
from datetime import date, timedelta
import re, pandas as pd, importlib.util, math

# --------- PARAMÈTRES À ADAPTER ---------
DAILY_DIR    = Path(r"G:\My Drive\14-DATA\TEC_DATA\Raw_Data\All_dcb_new")
MONTHLY_DIR  = Path(r"G:\My Drive\14-DATA\TEC_DATA\Raw_Data\All_dcb_new\CODE_DCB")  # fichiers .DCB (décompressés)
PARSER_PATH  = Path(r"G:\My Drive\14-DATA\TEC_DATA\obs_nav_dcb_parsers\latest_bsx_dsb_dcb_sat_c2w-c1c_parser_corr.py")
OUT_CSV      = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\NEW_all_dcb_master.csv")
START, END   = date(2015,10,1), date(2025,9,26)
PREFER_ORDER = ("DSB","DCB")   # ordre de préférence pour les lignes SINEX
# ----------------------------------------

# --- charger le parseur fourni ---
spec = importlib.util.spec_from_file_location("bsx_mod", str(PARSER_PATH))
if spec is None or spec.loader is None:
    raise FileNotFoundError(f"Parseur introuvable: {PARSER_PATH}")
bsx_mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(bsx_mod)  # type: ignore

# --- utilitaires ---
def yymm_from_ym(y:int, m:int) -> str:
    return f"{y%100:02d}{m:02d}"

def monthlies_paths(y:int, m:int):
    yymm = yymm_from_ym(y, m)
    p1c1 = MONTHLY_DIR / f"P1C1{yymm}.DCB"
    p1p2 = MONTHLY_DIR / f"P1P2{yymm}_ALL.DCB"
    return p1c1 if p1c1.exists() else None, p1p2 if p1p2.exists() else None

def cache_monthly(cache: dict, y:int, m:int):
    key = (y, m)
    if key in cache: 
        return cache[key]
    p1c1_path, p1p2_path = monthlies_paths(y, m)
    m1 = bsx_mod.parse_code_monthly_table(str(p1c1_path)) if p1c1_path else None
    m2 = bsx_mod.parse_code_monthly_table(str(p1p2_path)) if p1p2_path else None
    cache[key] = (m1, m2, p1c1_path, p1p2_path)
    return cache[key]

FN_CANDIDATES = lambda y,doy: [
    f"CAS0OPSRAP_{y}{doy:03d}0000_01D_01D_DCB.BIA",
    f"CAS0MGXRAP_{y}{doy:03d}0000_01D_01D_DCB.BSX",
    f"GFZ0OPSRAP_{y}{doy:03d}0000_01D_01D_DCB.BIA",
]

def pick_daily_file(y:int, doy:int) -> Path|None:
    # cherche .BIA/.BSX ou .gz correspondants
    for stem in FN_CANDIDATES(y, doy):
        p = DAILY_DIR / stem
        if p.exists(): return p
        gz = p.with_suffix(p.suffix + ".gz")
        if gz.exists(): return gz
    return None

def parse_year_doy_from_name(name: str):
    m = re.search(r"_(\d{4})(\d{3})\d{5}_", name)
    if m: return int(m.group(1)), int(m.group(2))
    m2 = re.search(r"_(\d{4})(\d{3})_", name)
    if m2: return int(m2.group(1)), int(m2.group(2))
    return None, None

def build_from_monthlies(month_p1c1: dict|None, month_p1p2: dict|None):
    """Retourne [(prn, val_m, source)] en utilisant (P1-C1) - (P1-P2) si dispo."""
    rows = []
    if not (month_p1c1 and month_p1p2):
        return rows
    common = set(month_p1c1.keys()) & set(month_p1p2.keys())
    for prn in sorted(common):
        val = month_p1c1[prn] - month_p1p2[prn]  # = C2W-C1C
        rows.append((prn, float(val), "derived:monthly(P1-C1)-(P1-P2)"))
    return rows

# --- boucle jours ---
records = []
monthly_cache = {}

cur = START
while cur <= END:
    y, doy = cur.year, int(cur.strftime("%j"))
    m = cur.month
    daily_path = pick_daily_file(y, doy)

    # charge mensuels du mois (serviront au besoin)
    month_p1c1, month_p1p2, p1c1_path, p1p2_path = cache_monthly(monthly_cache, y, m)

    if daily_path:
        # 1) essaye direct/inversé/derivé SINEX
        try:
            rows = bsx_mod.parse_sinex_bias(str(daily_path))
        except Exception as e:
            rows = []
        tbl = bsx_mod.build_table(rows, prefer=PREFER_ORDER, monthly_p1c1=month_p1c1, monthly_p1p2=None)

        if not tbl:
            # 3) pas de lignes SINEX: bascule mensuels seuls (si dispos)
            tbl = build_from_monthlies(month_p1c1, month_p1p2)

        # remplir enregistrements
        for prn, val_m, src in tbl:
            records.append({
                "date": cur.isoformat(),
                "year": y,
                "doy": doy,
                "prn": prn,
                "C2W-C1C_sat_m": val_m,
                "source": src,
                "daily_file": daily_path.name if daily_path else "",
                "monthly_p1c1": p1c1_path.name if p1c1_path else "",
                "monthly_p1p2": p1p2_path.name if p1p2_path else "",
            })
    else:
        # 2/3) aucun journalier: construire uniquement depuis mensuels
        tbl = build_from_monthlies(month_p1c1, month_p1p2)
        for prn, val_m, src in tbl:
            records.append({
                "date": cur.isoformat(),
                "year": y,
                "doy": doy,
                "prn": prn,
                "C2W-C1C_sat_m": val_m,
                "source": src,
                "daily_file": "",
                "monthly_p1c1": p1c1_path.name if p1c1_path else "",
                "monthly_p1p2": p1p2_path.name if p1p2_path else "",
            })

    cur += timedelta(days=1)

# --- DataFrame + export ---
df = pd.DataFrame.from_records(records,
       columns=["date","year","doy","prn","C2W-C1C_sat_m","source","daily_file","monthly_p1c1","monthly_p1p2"])
df.to_csv(OUT_CSV, index=False)
print(f"rows: {len(df)}  | écrit: {OUT_CSV}")

# aperçu
display(df.head(20))
print("\nComptage par source:")
print(df["source"].value_counts(dropna=False))


Generate a navigation master dataframe

In [ ]:
# =========================
# MINI-BATCH NAV RINEX2 → DataFrame unique (option CSV)
#   - Parcourt *.??n / *.n (majuscules/minuscules)
#   - Utilise le parseur nav_rinex2_to_df.py
#   - Sortie: DataFrame "df_nav" (et optionnellement un CSV)
# =========================
from pathlib import Path
import importlib.util
import pandas as pd

# >>>>>>>>> ADAPTE ICI <<<<<<<<<
IN_DIR = Path(r"G:\My Drive\14-DATA\TEC_DATA\Raw_Data\All_RINEX")           # dossier contenant les .YYn / .n
PARSER = Path(r"G:\My Drive\14-DATA\TEC_DATA\obs_nav_dcb_parsers\nav_rinex2_to_df.py")     # chemin vers ton parseur
RECURSIVE = False                                         # True = inclut les sous-dossiers
WRITE_CSV = True                                        # True pour écrire un CSV
OUT_CSV = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\NEW_all_nav_master.csv")  # chemin CSV si WRITE_CSV=True        
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# -- Charger le parseur comme module --
spec = importlib.util.spec_from_file_location("nav_mod", str(PARSER))
if spec is None or spec.loader is None:
    raise FileNotFoundError(f"Impossible de charger le parseur: {PARSER}")
nav_mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(nav_mod)  # type: ignore

# -- Collecte des fichiers NAV --
if not IN_DIR.is_dir():
    raise SystemExit(f"Dossier introuvable: {IN_DIR}")
patterns = ["*.??n", "*.n", "*.??N", "*.N"]
files = []
for pat in patterns:
    files += list(IN_DIR.rglob(pat) if RECURSIVE else IN_DIR.glob(pat))
files = sorted({p.resolve() for p in files if p.is_file()})
print(f"Fichiers NAV trouvés: {len(files)}")

# -- Parse & agrégation --
rows_all = []
for p in files:
    try:
        ephs = nav_mod.parse_rinex2_nav(str(p))  # liste de dicts (voir parser)
        if not ephs:
            print(f"⚠️ Aucun éphéméride lu dans: {p.name}")
            continue
        for e in ephs:
            d = dict(e)
            # convertir datetime -> ISO 8601 comme dans l’exemple
            if "toc" in d and hasattr(d["toc"], "isoformat"):
                d["toc"] = d["toc"].isoformat()
            d["nav_file"] = p.name  # traçabilité
            rows_all.append(d)
    except Exception as ex:
        print(f"⚠️ Erreur sur {p.name}: {ex}")

# -- DataFrame final (ordre de colonnes calqué sur le parseur) --
cols_order = [
    "prn","toc","af0","af1","af2",
    "IODE","Crs","d_n","M0","Cuc","e","Cus","sqrtA",
    "Toe","Cic","Omega0","Cis","i0","Crc","omega","OMEGA_DOT",
    "IDOT","week","sv_accuracy","sv_health","Tgd","IODC","TxTime","fit_int",
    "nav_file"
]
df_nav = pd.DataFrame(rows_all)
# garder uniquement les colonnes attendues (si certaines manquent/NaN, ce n’est pas bloquant)
df_nav = df_nav[[c for c in cols_order if c in df_nav.columns]].copy()

# -- Tri & déduplication douce (une ligne par PRN & toc) --
if {"prn","toc"}.issubset(df_nav.columns):
    before = len(df_nav)
    df_nav.drop_duplicates(subset=["prn","toc"], inplace=True)
    after = len(df_nav)
    if after != before:
        print(f"Déduplication: {before} -> {after} lignes (clé: prn+toc)")
df_nav.sort_values(["prn","toc"], inplace=True, ignore_index=True)

print(f"Lignes agrégées: {len(df_nav)}")

# -- Affichage interactif (si dispo), sinon head() --
try:
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user("NAV_master", df_nav)
except Exception:
    display(df_nav.head(20))

# -- Option: écrire un CSV unique sur disque --
if WRITE_CSV:
    OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
    df_nav.to_csv(OUT_CSV, index=False)
    print(f"✅ CSV écrit: {OUT_CSV}")


Generate Parquet files from RINEX files

In [ ]:
# Cell 8: Generate Parquet (and .csv) files from obs rinex files
# Cette cellule est destinée à générer des fichiers csv et parquet à partir des fichiers rinex obs (.yyo)
# contenant les informations necessaires pour le calcul du vtec (en utilisant le fichier dcb matser et nav master). Le calcul final
# fait appel au notebook 00_Generate_vtec_stats_csv_from_master_files_vf.ipynb
# --------------------------

from pathlib import Path
import pandas as pd
from rinex2_obs_lean import parse_rinex2_obs_lean

# >>> adapte le chemin dossier des .??o
IN_DIR = Path(r"G:\My Drive\14-DATA\TEC_DATA\Raw_Data\All_RINEX\rnx_from_raw")   # contient *.??o / *.o
RECURSIVE = True                          # True si sous-dossiers à parcourir
WRITE_PARQUET = True                    # True si tu veux aussi enregistrer un fichier léger
OUT_MASTER_CSV = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\All_Obs_master\NEW_obs_master_1200_1477.csv")
OUT_PARQUET = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\All_Obs_master\Parquet\NEW_OBS_lean_1200_1477.parquet")

patterns = ["*.??o","*.o","*.??O","*.O"]
files = []
for pat in patterns:
    files += list(IN_DIR.rglob(pat) if RECURSIVE else IN_DIR.glob(pat))
files = sorted({p.resolve() for p in files if p.is_file()})
print(f"Fichiers trouvés: {len(files)}")

dfs = []
for i, p in enumerate(files[1200:1478], 1): #c'set dans cette ligne où on doit pointer vers les fichiers .yyo en question!
    try:
        df = parse_rinex2_obs_lean(str(p))
        if not df.empty:
            df["source_file"] = p.name  # traçabilité (catégorie)
            df["source_file"] = df["source_file"].astype("category")
            dfs.append(df)
        else:
            print(f"  - {p.name}: aucun C1/P2 lisible")
    except Exception as e:
        print(f"⚠️ {p.name}: {e}")

if dfs:
    obs_lean = pd.concat(dfs, ignore_index=True)
    obs_lean.sort_values(["time","prn"], inplace=True, ignore_index=True)
    print(f"Lignes totalisées: {len(obs_lean)}")
else:
    obs_lean = pd.DataFrame(columns=["time","prn","P2_minus_C1_m","source_file"])
    print("Aucune donnée.")

# Affichage rapide
display(obs_lean.head(20))

# Option: sauvegarde ultra-rapide & compacte
if WRITE_PARQUET and not obs_lean.empty:
    OUT_PARQUET.parent.mkdir(parents=True, exist_ok=True)
    obs_lean.to_csv(OUT_MASTER_CSV, index=False)
    obs_lean.to_parquet(OUT_PARQUET, index=False)
    print(f"✅ Parquet écrit: {OUT_PARQUET}")


Generate Receiver bias per day over a period: WLS vs Minimum-Scalloping

In [ ]:
# Receiver bias per day over a period: WLS vs Minimum-Scalloping (self-contained)

import math, numpy as np, pandas as pd
from pathlib import Path

# ===== CONFIG =====
#OBS_DIR   = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\All_Obs_master\Parquet")   # parquet/csv: time, prn, P2_minus_C1_m
#NAV_FILE  = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\NEW_all_nav_master.csv")   # broadcast ephemeris CSV
#DCB_FILE  = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\NEW_all_dcb_master.csv")   # date|year+doy, prn, C2W-C1C_sat_m
#OUT_DRX_CSV = Path(r"G:\My Drive\14-DATA\TEC_DATA\Drx_WLS_vs_MS_daily.csv")

OBS_DIR   = Path("/home/mk-laptop/Downloads/TEC_DATA/Master/All_Obs_master/Parquet/")   # parquet/csv: time, prn, P2_minus_C1_m
NAV_FILE  = Path("/home/mk-laptop/Downloads/TEC_DATA/Master/NEW_all_nav_master.csv")   # broadcast ephemeris CSV
DCB_FILE  = Path("/home/mk-laptop/Downloads/TEC_DATA/Master/NEW_all_dcb_master.csv")   # date|year+doy, prn, C2W-C1C_sat_m
OUT_DRX_CSV = Path("/home/mk-laptop/Downloads/TEC_DATA/Drx_WLS_vs_MS_daily.csv")


DATE_START = "2015-10-01"
DATE_END   = "2025-09-26"

RX_XYZ = (5411106.3084, -747628.5974, 3286931.3223)  # station ECEF (m)

ELEV_MIN_DEG   = 10.0
H_IONO         = 350_000.0
R_EARTH        = 6378137.0
MAX_RESID_M    = 5.0
MAX_VTEC_TECU  = 200.0
MERGE_TOL      = "2h"   # obs-ephemeris nearest tolerance

# GPS L1/L2
f1 = 1575.42e6; f2 = 1227.60e6
K_m_per_TECU = 40.3*(1.0/f2**2 - 1.0/f1**2)  # ≈0.105 m/TECU

# ===== NAV / DCB =====
def load_nav(path: Path) -> pd.DataFrame:
    nav = pd.read_csv(path)
    nav["prn"] = nav["prn"].str.upper().str.strip()
    nav["toc"] = pd.to_datetime(nav["toc"], utc=True, errors="coerce")
    nav["ephem_time"] = (pd.to_datetime("1980-01-06", utc=True)
                         + pd.to_timedelta(nav["week"]*7, unit="D")
                         + pd.to_timedelta(nav["Toe"], unit="s"))
    keep = ["prn","ephem_time","week","Toe","sqrtA","d_n","e","M0","Cuc","Cus",
            "Crc","Crs","Cic","Cis","i0","IDOT","Omega0","OMEGA_DOT","omega"]
    return nav[keep].sort_values(["prn","ephem_time"]).reset_index(drop=True)

def load_dcb(path: Path) -> pd.DataFrame:
    dcb = pd.read_csv(path)
    dcb["prn"] = dcb["prn"].str.upper().str.strip()
    if "date" in dcb.columns:
        dcb["dcb_date"] = pd.to_datetime(dcb["date"], utc=True).dt.floor("D")
    else:
        base = pd.to_datetime(dcb["year"].astype(int).astype(str), format="%Y", utc=True)
        dcb["dcb_date"] = (base + pd.to_timedelta(dcb["doy"].astype(int)-1, unit="D")).dt.floor("D")
    dcb = dcb.rename(columns={"C2W-C1C_sat_m":"sat_dcb_m"})
    return dcb[["prn","dcb_date","sat_dcb_m"]]

# ===== OBS =====
def _read_day_slice(fp: Path, t0: pd.Timestamp, t1: pd.Timestamp) -> pd.DataFrame:
    cols = ["time","prn","P2_minus_C1_m"]
    if fp.suffix.lower()==".parquet":
        try:
            df = pd.read_parquet(fp, columns=cols,
                                 filters=[("time", ">=", t0), ("time", "<", t1)])
        except Exception:
            df = pd.read_parquet(fp, columns=cols)
            df["time"] = pd.to_datetime(df["time"], utc=True)
            df = df[(df["time"]>=t0)&(df["time"]<t1)]
    else:
        df = pd.read_csv(fp, usecols=cols, parse_dates=["time"])
        df["time"] = pd.to_datetime(df["time"], utc=True)
        df = df[(df["time"]>=t0)&(df["time"]<t1)]
    return df

def stack_obs_for_day(t0: pd.Timestamp) -> pd.DataFrame | None:
    t1 = t0 + pd.Timedelta(days=1)
    files = sorted([*OBS_DIR.glob("*.parquet"), *OBS_DIR.glob("*.csv")])
    parts=[]
    for fp in files:
        df = _read_day_slice(fp, t0, t1)
        if df is not None and len(df): parts.append(df)
    if not parts: return None
    obs = pd.concat(parts, ignore_index=True)
    obs["prn"]  = obs["prn"].str.upper().str.strip()
    obs["time"] = pd.to_datetime(obs["time"], utc=True)
    obs = obs.dropna(subset=["time","prn","P2_minus_C1_m"])
    return obs.sort_values(["prn","time"]).reset_index(drop=True)

# ===== Geometry =====
def ecef_to_geodetic(x,y,z):
    a=6378137.0; f=1/298.257223563; e2=f*(2-f); b=a*(1-f); ep2=(a*a-b*b)/(b*b)
    r=math.hypot(x,y); E2=a*a-b*b; F=54*b*b*z*z; G=r*r+(1-e2)*z*z-e2*E2
    c=(e2*e2*F*r*r)/(G*G*G); s=(1+c+math.sqrt(c*c+2*c))**(1/3)
    P=F/(3*(s+1/s+1)**2*G*G); Q=math.sqrt(1+2*e2*e2*P)
    r0=-(P*e2*r)/(1+Q)+math.sqrt(0.5*a*a*(1+1/Q)-(P*(1-e2)*z*z)/(Q*(1+Q))-0.5*P*r*r)
    U=math.sqrt((r-e2*r0)**2+z*z); V=math.sqrt((r-e2*r0)**2+(1-e2)*z*z)
    Z0=b*b*z/(a*V); h=U*(1-b*b/(a*V)); lat=math.atan2(z+ep2*Z0, r); lon=math.atan2(y,x)
    return lat,lon,h

def elevation_deg(rx_xyz, sat_xyz):
    x,y,z = rx_xyz
    lat,lon,_ = ecef_to_geodetic(x,y,z)
    sl,cl = math.sin(lat), math.cos(lat); slon,clon = math.sin(lon), math.cos(lon)
    R = np.array([[-slon,        clon,       0.0],
                  [-clon*sl, -slon*sl,      cl ],
                  [ clon*cl,  slon*cl,      sl ]])
    d = np.array([sat_xyz[0]-x, sat_xyz[1]-y, sat_xyz[2]-z])
    e,n,u = R @ d
    return math.degrees(math.atan2(u, math.hypot(e,n)))

def sat_ecef_from_navrow(t_utc, rnav):
    MU=3.986005e14; OMEGA_E=7.2921151467e-5
    t = pd.Timestamp(t_utc).tz_convert("UTC")
    toe_time = pd.to_datetime(rnav["ephem_time"], utc=True)
    tk = (t - toe_time).total_seconds()
    half=302400.0
    if tk>half: tk-=2*half
    if tk<-half: tk+=2*half
    A=float(rnav["sqrtA"])**2; dn=float(rnav["d_n"]); ecc=float(rnav["e"]); M0=float(rnav["M0"])
    Cuc=float(rnav["Cuc"]); Cus=float(rnav["Cus"]); Crc=float(rnav["Crc"]); Crs=float(rnav["Crs"])
    Cic=float(rnav["Cic"]); Cis=float(rnav["Cis"]); i0=float(rnav["i0"]); IDOT=float(rnav["IDOT"])
    OMG0=float(rnav["Omega0"]); OMG_DOT=float(rnav["OMEGA_DOT"]); omg=float(rnav["omega"]); toe_s=float(rnav["Toe"])
    n0=math.sqrt(MU/A**3); n=n0+dn; M=M0+n*tk
    E=M
    for _ in range(12):
        dE=(M+ecc*math.sin(E)-E)/(1-ecc*math.cos(E)); E+=dE
        if abs(dE)<1e-13: break
    v=math.atan2(math.sqrt(1-ecc*ecc)*math.sin(E), math.cos(E)-ecc)
    phi=v+omg
    du=Cuc*math.cos(2*phi)+Cus*math.sin(2*phi)
    dr=Crc*math.cos(2*phi)+Crs*math.sin(2*phi)
    di=Cic*math.cos(2*phi)+Cis*math.sin(2*phi)
    u=phi+du; r=A*(1-ecc*math.cos(E))+dr; i=i0+IDOT*tk+di
    x_orb=r*math.cos(u); y_orb=r*math.sin(u)
    OMG=OMG0+(OMG_DOT-OMEGA_E)*tk - OMEGA_E*toe_s
    cosO, sinO = math.cos(OMG), math.sin(OMG); cosi, sini = math.cos(i), math.sin(i)
    x= x_orb*cosO - y_orb*cosi*sinO
    y= x_orb*sinO + y_orb*cosi*cosO
    z= y_orb*sini
    return x,y,z

def merge_nav(obs_day: pd.DataFrame, nav: pd.DataFrame, tol: str | None) -> pd.DataFrame:
    out=[]
    for prn, left in obs_day.groupby("prn", sort=False):
        right = nav[nav["prn"]==prn]
        if right.empty: continue
        left  = left.sort_values("time")
        right = right.sort_values("ephem_time")
        kw=dict(direction="nearest")
        if tol: kw["tolerance"]=pd.Timedelta(tol)
        m = pd.merge_asof(left=left, right=right, left_on="time", right_on="ephem_time", **kw)
        if m is None or m.empty: continue
        if "prn_x" in m.columns and "prn" not in m.columns:
            m = m.rename(columns={"prn_x":"prn"})
        if "prn_y" in m.columns:
            m = m.drop(columns=["prn_y"])
        out.append(m)
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

# ===== Mapping + DCB =====
def mapping_M(el_deg, Re=R_EARTH, H=H_IONO):
    el = np.radians(el_deg)
    arg = (Re*np.cos(el))/(Re+H)
    M = 1.0/np.sqrt(1.0 - np.clip(arg*arg, 0.0, 0.9999))
    return np.clip(M, 1.0, 20.0)

def attach_sat_dcb(m: pd.DataFrame, dcb_day: pd.DataFrame) -> pd.DataFrame:
    mm = m.merge(dcb_day, on="prn", how="inner").dropna(subset=["sat_dcb_m"])
    if mm.empty: return mm
    mm["y_m"] = mm["P2_minus_C1_m"] - mm["sat_dcb_m"]
    return mm

def mapping_block(df: pd.DataFrame):
    df["M"] = mapping_M(df["elev_deg"].values)
    df["P"] = K_m_per_TECU * df["M"].values
    df["w"] = 1.0/(df["M"].values**2)
    return df

# ===== WLS (closed form, 2 passes) =====
def solve_closed_form(df: pd.DataFrame):
    df = df.dropna(subset=["y_m","M","P","w","time"])
    if df.empty: return None, None, None
    d = df.assign(wP=df["w"]*df["P"], wP2=df["w"]*(df["P"]**2),
                  wy=df["w"]*df["y_m"], wPy=df["w"]*df["P"]*df["y_m"])
    G = d.groupby("time", sort=True)[["w","wP","wP2","wPy"]].sum()
    S0,S1,S2,b_t = G["w"],G["wP"],G["wP2"],G["wPy"]
    b0 = d["wy"].sum()
    valid = (S2>0) & (S0>=3)
    if not valid.any(): return None, None, None
    S0,S1,S2,b_t = S0[valid],S1[valid],S2[valid],b_t[valid]
    num = b0 - np.sum(S1*b_t/S2); den = np.sum(S0) - np.sum((S1**2)/S2)
    if not np.isfinite(den) or den==0: return None, None, None
    D = num/den; V_t = (b_t - S1*D)/S2
    times = pd.DatetimeIndex(G.index[valid])
    if times.tz is None: times = times.tz_localize("UTC")
    times = times.tz_convert("UTC").rename(None)
    vt = pd.DataFrame({"time": times.to_numpy(copy=False),
                       "VTEC_TECU": np.asarray(V_t, dtype=float)})
    vt.sort_values(by="time", inplace=True, kind="mergesort")
    vt.reset_index(drop=True, inplace=True)
    return float(D), vt, {"N_obs": int(len(df))}

def wls_two_pass(df: pd.DataFrame, max_resid_m=MAX_RESID_M):
    D1, vt1, _ = solve_closed_form(df)
    if D1 is None: return None, None, {}
    df["time"]  = pd.to_datetime(df["time"],  utc=True)
    vt1["time"] = pd.to_datetime(vt1["time"], utc=True)
    vt1_idx = vt1.set_index("time")["VTEC_TECU"]
    d1 = df.join(vt1_idx, on="time", how="inner")
    pred = D1 + d1["P"].values * d1["VTEC_TECU"].values
    resid = d1["y_m"].values - pred
    keep = np.isfinite(resid) & (np.abs(resid) <= max_resid_m)
    d2 = d1.loc[keep].copy()
    D2, vt2, diag = solve_closed_form(d2)
    if D2 is None: return None, None, {}
    vt2 = vt2[(vt2["VTEC_TECU"]>=0) & (vt2["VTEC_TECU"]<=MAX_VTEC_TECU)].copy()
    diag = {
        "N_obs_pass1": int(len(d1)),
        "N_obs_pass2": int(len(d2)),
        "N_epochs":    int(len(vt2)),
        "Nsat_mean":   float(d2.groupby("time").size().mean()) if len(d2) else np.nan,
    }
    return float(D2), vt2, diag

# ===== Minimum-scalloping =====
def _scalloping_score(D, df, night_only=True):
    vtec_i = (df["y_m"].values - D) / df["P"].values
    tmp = pd.DataFrame({"time": pd.to_datetime(df["time"], utc=True), "vtec": vtec_i})
    if night_only:
        lt = tmp["time"].dt.tz_convert("Africa/Casablanca").dt.hour
        tmp = tmp[(lt<=4) | (lt>=22)]
        if tmp.empty: return np.inf
    def mad(a):
        m = np.median(a); return np.median(np.abs(a - m))
    disp = tmp.groupby("time")["vtec"].apply(mad)
    disp = disp[np.isfinite(disp)]
    return float(np.median(disp)) if len(disp) else np.inf

def minimum_scalloping_D(df, D_init=None, span_m=10.0, step_m=0.1, night_only=True):
    if D_init is None:
        D0,_,_ = solve_closed_form(df)
        if D0 is None: return None, {}
        D_init = D0
    grid = np.arange(D_init - span_m, D_init + span_m + step_m, step_m)
    scores = [ _scalloping_score(D, df, night_only=night_only) for D in grid ]
    k = int(np.argmin(scores))
    return float(grid[k]), {"MS_score_min": float(scores[k]), "MS_grid_step": float(step_m)}

# ===== One day =====
def drx_for_one_day(day_str, elev_min_deg=ELEV_MIN_DEG, night_only=True,
                    nav=None, dcb=None) -> dict | None:
    if nav is None: nav = load_nav(NAV_FILE)
    if dcb is None: dcb = load_dcb(DCB_FILE)
    t0 = pd.Timestamp(day_str, tz="UTC").floor("D")

    obs_day = stack_obs_for_day(t0)
    if obs_day is None or obs_day.empty: return None

    m = merge_nav(obs_day, nav, tol=MERGE_TOL)
    if m is None or m.empty: return None

    dcb_day = dcb[dcb["dcb_date"]==t0.floor("D")][["prn","sat_dcb_m"]]
    if dcb_day.empty: return None

    sv = np.vstack([sat_ecef_from_navrow(row["time"], row) for _,row in m.iterrows()])
    m["sv_x"],m["sv_y"],m["sv_z"] = sv[:,0], sv[:,1], sv[:,2]
    m["elev_deg"] = [elevation_deg(RX_XYZ, (x,y,z)) for x,y,z in zip(m["sv_x"],m["sv_y"],m["sv_z"])]

    m = attach_sat_dcb(m, dcb_day)
    if m.empty: return None

    m = m[m["elev_deg"] >= float(elev_min_deg)].copy()
    m = mapping_block(m)

    D_wls, vt, diag = wls_two_pass(m, max_resid_m=MAX_RESID_M)
    if D_wls is None: return None

    D_ms, info_ms = minimum_scalloping_D(m, D_init=D_wls, span_m=10.0, step_m=0.1, night_only=True)
    delta = (D_ms - D_wls) if D_ms is not None else np.nan

    return {
        "date": t0.date(),
        "D_rx_WLS_m": float(D_wls),
        "D_rx_MS_m": float(D_ms) if D_ms is not None else np.nan,
        "delta_MS_minus_WLS_m": float(delta) if np.isfinite(delta) else np.nan,
        "N_obs_pass1": diag.get("N_obs_pass1", np.nan),
        "N_obs_pass2": diag.get("N_obs_pass2", np.nan),
        "N_epochs":    diag.get("N_epochs",    np.nan),
        "Nsat_mean":   diag.get("Nsat_mean",   np.nan),
        "MS_score_min": info_ms.get("MS_score_min", np.nan)
    }

# ===== Period runner =====
def run_period(date_start=DATE_START, date_end=DATE_END, out_csv=OUT_DRX_CSV,
               elev_min_deg=ELEV_MIN_DEG):
    nav = load_nav(NAV_FILE)
    dcb = load_dcb(DCB_FILE)

    days = pd.date_range(pd.Timestamp(date_start, tz="UTC"),
                         pd.Timestamp(date_end,   tz="UTC"),
                         freq="D")
    if out_csv.exists(): out_csv.unlink()
    rows = []

    for t0 in days:
        day_txt = str(t0.date())
        try:
            rec = drx_for_one_day(day_txt, elev_min_deg=elev_min_deg, night_only=True,
                                  nav=nav, dcb=dcb)
            if rec is None:
                print(f"⏭️ {day_txt}: no result (missing obs/nav/dcb/solvability).")
                continue
            rows.append(rec)
            pd.DataFrame([rec]).to_csv(out_csv, mode="a", header=not out_csv.exists(), index=False)
            print(f"✅ {day_txt}  WLS={rec['D_rx_WLS_m']:.3f}  MS={rec['D_rx_MS_m']:.3f}  Δ={rec['delta_MS_minus_WLS_m']:.3f}")
        except Exception as e:
            print(f"⚠️ {day_txt}: error → {e}")

    if rows:
        df = pd.DataFrame(rows).sort_values("date")
        df.to_csv(out_csv, index=False)
        print(f"\n→ Saved: {out_csv}")
    else:
        print("No rows produced.")

# ===== Run =====
if __name__ == "__main__":
    pd.set_option("display.width", 160)
    pd.set_option("display.max_columns", 20)
    run_period()


Generate a vtec_gim dataframe

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import io, subprocess
from datetime import date, datetime, timedelta, timezone

# ====== PARAMÈTRES ======
DEC_DIR   = Path(r"G:\My Drive\14-DATA\TEC_DATA\Raw_Data\IONEX\IONEX_decompressed")  # dossier UNIQUE des IONEX décompressés (.i / .INX)
OUT_CSV   = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\VTEC_GIM_30min_20151001_20250926.csv")
STA_LAT   = 31.206
STA_LON   = -7.866
START     = date(2015, 10, 1)
END       = date(2025, 9, 26)
# ========================

# --- décompression lecture (supporte aussi .Z/.gz si restés par erreur) ---
def _open_text(p: Path) -> io.StringIO:
    p = Path(p)
    if not p.exists(): raise FileNotFoundError(p)
    ext = p.suffix.lower()
    if ext == ".gz":
        out = subprocess.run(["gzip", "-dc", str(p)], capture_output=True)
        if out.returncode != 0: raise RuntimeError("gzip -dc failed")
        return io.StringIO(out.stdout.decode("ascii","ignore"))
    if ext == ".z":
        # essaye gzip d’abord
        gz = subprocess.run(["gzip", "-dc", str(p)], capture_output=True)
        if gz.returncode == 0 and gz.stdout:
            return io.StringIO(gz.stdout.decode("ascii","ignore"))
        # sinon 7z
        sz = subprocess.run(["7z", "e", "-so", str(p)], capture_output=True)
        if sz.returncode != 0: raise RuntimeError("7z -so failed")
        return io.StringIO(sz.stdout.decode("ascii","ignore"))
    return io.StringIO(p.read_text(encoding="ascii", errors="ignore"))

def ionex_first_epoch_date(path: Path):
    try:
        f = _open_text(path)
    except Exception:
        return None
    for _ in range(400):
        line = f.readline()
        if not line: break
        if "EPOCH OF FIRST MAP" in line:
            yr, mo, dy, hh, mm, ss = map(int, line[:60].split()[:6])
            return datetime(yr, mo, dy, hh, mm, ss, tzinfo=timezone.utc).date()
        if "END OF HEADER" in line:
            break
    return None

def read_ionex(path: Path):
    f = _open_text(path)
    exp = -1
    lat1=lat2=dlat=lon1=lon2=dlon=None
    while True:
        line = f.readline()
        if not line: raise ValueError("Header IONEX incomplet")
        if "EXPONENT" in line:
            s = line[:8].strip()
            exp = int(s) if s else -1
        if "LAT1 / LAT2 / DLAT" in line:
            lat1,lat2,dlat = map(float, line[:60].split()[:3])
        if "LON1 / LON2 / DLON" in line:
            lon1,lon2,dlon = map(float, line[:60].split()[:3])
        if "END OF HEADER" in line:
            break
    if None in (lat1,lat2,dlat,lon1,lon2,dlon): raise ValueError("Grille absente")
    nlat = int(round((lat2-lat1)/dlat))+1
    nlon = int(round((lon2-lon1)/dlon))+1
    lats = np.linspace(lat1, lat2, nlat)
    lons = np.linspace(lon1, lon2, nlon)

    times, maps = [], []
    while True:
        line = f.readline()
        if not line: break
        if "START OF TEC MAP" in line:
            # epoch
            line = f.readline()
            while line and "EPOCH OF CURRENT MAP" not in line:
                line = f.readline()
            if not line: break
            yr,mo,dy,hh,mm,ss = map(int, line[:60].split()[:6])
            t = pd.Timestamp(datetime(yr,mo,dy,hh,mm,ss, tzinfo=timezone.utc))
            tec = np.full((nlat,nlon), np.nan)
            bad = False
            for _ in range(nlat):
                hdr = f.readline()
                if not hdr or "LAT/LON1/LON2/DLON/H" not in hdr:
                    bad=True; break
                vals=[]
                while len(vals) < nlon:
                    data = f.readline()
                    if not data or ("START OF" in data) or ("END OF" in data) or ("LAT/LON1" in data):
                        bad=True; break
                    chunks = [data[i:i+5] for i in range(0, len(data.rstrip()), 5)]
                    for c in chunks:
                        c = c.strip().upper()
                        if c=="" or c=="9999": vals.append(np.nan)
                        else:
                            try: vals.append(float(c)*(10.0**exp))
                            except: vals.append(np.nan)
                        if len(vals)==nlon: break
                if bad: break
                if len(vals)<nlon: vals += [np.nan]*(nlon-len(vals))
                tec[len(times) if False else _ , :]  # dummy no-op to keep linters calm
                tec[_ , :] = vals  # uses loop index positionally
            if bad:
                # consume until end of map
                x = hdr
                while x:
                    if "END OF TEC MAP" in x: break
                    x = f.readline()
                continue
            times.append(t)
            maps.append(tec)
    if not maps: raise ValueError("Aucune carte TEC")
    TEC = np.stack(maps, axis=0)
    if lons.min()>=0 and lons.max()>180:
        order = np.argsort(((lons+180)%360)-180)
        lons = (((lons+180)%360)-180)[order]
        TEC  = TEC[:,:,order]
    if lats[0]>lats[-1]:
        lats = lats[::-1]; TEC = TEC[:, ::-1, :]
    times = pd.to_datetime(times, utc=True)
    return times, lats, lons, TEC

def bilinear(lats, lons, grid, y, x):
    i = np.searchsorted(lats, y) - 1
    j = np.searchsorted(lons, x) - 1
    i = np.clip(i, 0, len(lats)-2); j = np.clip(j, 0, len(lons)-2)
    y1,y2 = lats[i], lats[i+1]; x1,x2 = lons[j], lons[j+1]
    Q11,Q12,Q21,Q22 = grid[i,j], grid[i,j+1], grid[i+1,j], grid[i+1,j+1]
    if (x2-x1)==0 or (y2-y1)==0: return float(Q11)
    wx = (x-x1)/(x2-x1); wy = (y-y1)/(y2-y1)
    return float((1-wx)*(1-wy)*Q11 + wx*(1-wy)*Q12 + (1-wx)*wy*Q21 + wx*wy*Q22)

def to_utc_index(x) -> pd.DatetimeIndex:
    idx = pd.DatetimeIndex(x)
    if idx.tz is None: idx = idx.tz_localize("UTC")
    else: idx = idx.tz_convert("UTC")
    return idx.sort_values().unique()

def make_30min_grid(day: date) -> pd.DatetimeIndex:
    t0 = pd.Timestamp(day, tz="UTC")
    return pd.date_range(t0, periods=48, freq="30min")

# --- fenêtre produit par date ---
def product_window(day: date) -> str:
    # renvoie 'OLD', 'OPSFIN', ou 'OPSRAP'
    if day <= date(2022, 11, 27):  # 2022-DOY330 inclus -> OLD
        return "OLD"
    if day <= date(2025, 9, 20):   # 2025-DOY263 inclus -> OPSFIN
        return "OPSFIN"
    return "OPSRAP"                # après -> RAPID

# --- sélection du fichier IONEX décompressé dans DEC_DIR ---
def pick_ionex_for_day(day: date) -> Path | None:
    yy = f"{day.year%100:02d}"; doy = f"{int(pd.Timestamp(day).strftime('%j')):03d}"
    mode = product_window(day)

    # OLD: codgDDD0.yyi (min/maj)
    if mode == "OLD":
        cand = [f"codg{doy}0.{yy}i", f"CODG{doy}0.{yy}I"]
        for name in cand:
            p = DEC_DIR / name
            if p.exists() and ionex_first_epoch_date(p) == day:
                return p
        # si plusieurs codg*.i présents, valider par entête
        for q in DEC_DIR.glob(f"codg*{yy}i"):
            if ionex_first_epoch_date(q) == day: return q
        for q in DEC_DIR.glob(f"CODG*{yy}I"):
            if ionex_first_epoch_date(q) == day: return q
        return None

    # Long names: préférer COD0OPSFIN, sinon COD0OPSRAP. On valide l'entête.
    patterns = []
    if mode == "OPSFIN":
        patterns = [f"COD0OPSFIN_*{day.year}{doy}*_GIM.INX",
                    f"COD0OPSRAP_*{day.year}{doy}*_GIM.INX"]
    else:  # OPSRAP window
        patterns = [f"COD0OPSRAP_*{day.year}{doy}*_GIM.INX",
                    f"COD0OPSFIN_*{day.year}{doy}*_GIM.INX"]

    for pat in patterns:
        for q in DEC_DIR.glob(pat):
            if ionex_first_epoch_date(q) == day:
                return q
    return None

def vtec_30min_from_ionex(ionex_path: Path, day: date, lat: float, lon: float) -> pd.Series:
    times, lats, lons, TEC = read_ionex(ionex_path)
    lon_match = lon
    if float(np.nanmin(lons)) >= 0 and float(np.nanmax(lons)) > 180:
        lon_match = (lon + 360) % 360
    vals = [bilinear(lats, lons, TEC[k], lat, lon_match) for k in range(len(times))]
    ser = pd.Series(vals, index=to_utc_index(times), name="vtec_gim")
    # interpolation temporelle vers la grille 30 min du jour
    t30 = make_30min_grid(day)
    union = ser.index.union(t30)
    ser30 = ser.reindex(union).interpolate("time").reindex(t30)
    return ser30

# ====== BOUCLE PRINCIPALE ======
all_rows = []
d = START
while d <= END:
    p = pick_ionex_for_day(d)
    if p is None:
        # jour manquant -> NaN
        t30 = make_30min_grid(d)
        all_rows.append(pd.DataFrame({"time": t30, "vtec_gim": np.nan}))
    else:
        ser30 = vtec_30min_from_ionex(p, d, STA_LAT, STA_LON)
        all_rows.append(pd.DataFrame({"time": ser30.index, "vtec_gim": ser30.values}))
    d += timedelta(days=1)

df = pd.concat(all_rows, axis=0, ignore_index=True)
df.sort_values("time", inplace=True, ignore_index=True)
df.to_csv(OUT_CSV, index=False)
print(f"écrit: {OUT_CSV}  |  lignes: {len(df)}")


Cellule produisant deux fichiers csv par technique WLS ou MS

In [ ]:
# === VTEC stats with fixed D_rx (MS & WLS) → four CSVs ===

import math, numpy as np, pandas as pd
from pathlib import Path

# ---------- CONFIG ----------
#OBS_DIR   = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\All_Obs_master\Parquet")
#NAV_FILE  = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\NEW_all_nav_master.csv")
#DCB_FILE  = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\NEW_all_dcb_master.csv")
#GIM_FILE  = Path(r"G:\My Drive\14-DATA\TEC_DATA\Master_files\VTEC_GIM_30min_20151001_20250926.csv")
#DRX_FILE  = Path(r"G:\My Drive\14-DATA\TEC_DATA\Drx_WLS_vs_MS_daily.csv")  # doit contenir: date, D_rx_MS_m, D_rx_WLS_m

OBS_DIR   = Path("/home/mk-laptop/Downloads/TEC_DATA/Master/All_Obs_master/Parquet/")   # parquet/csv: time, prn, P2_minus_C1_m
NAV_FILE  = Path("/home/mk-laptop/Downloads/TEC_DATA/Master/NEW_all_nav_master.csv")   # broadcast ephemeris CSV
DCB_FILE  = Path("/home/mk-laptop/Downloads/TEC_DATA/Master/NEW_all_dcb_master.csv")   # date|year+doy, prn, C2W-C1C_sat_m
DRX_FILE  = Path("/home/mk-laptop/Downloads/TEC_DATA/Drx_WLS_vs_MS_daily_cycle_slips.csv")
GIM_FILE  = Path("/home/mk-laptop/Downloads/TEC_DATA/Master/VTEC_GIM_30min_20151001_20250926.csv")


#OUT_30MIN_MS = Path(r"G:\My Drive\14-DATA\TEC_DATA\2015_2025_MS_VTEC_30min_stats.csv")
#OUT_30MIN_WL = Path(r"G:\My Drive\14-DATA\TEC_DATA\2015_2025_WLS_VTEC_30min_stats.csv")
#OUT_DAILY_MS = Path(r"G:\My Drive\14-DATA\TEC_DATA\2015_2025_MS_VTEC_daily_stats.csv")
#OUT_DAILY_WL = Path(r"G:\My Drive\14-DATA\TEC_DATA\2015_2025_WLS_VTEC_daily_stats.csv")
#OUT_MATCH    = Path(r"G:\My Drive\14-DATA\TEC_DATA\2015_2025_MS_WLS_match_rates.csv")

OUT_30MIN_MS = Path("/home/mk-laptop/Downloads/TEC_DATA/2015_2025_MS_CS_VTEC_30min_stats.csv")
OUT_30MIN_WL = Path("/home/mk-laptop/Downloads/TEC_DATA/2015_2025_WLS_CS_VTEC_30min_stats.csv")
OUT_DAILY_MS = Path("/home/mk-laptop/Downloads/TEC_DATA/2015_2025_MS_CS_VTEC_daily_stats.csv")
OUT_DAILY_WL = Path("/home/mk-laptop/Downloads/TEC_DATA/2015_2025_WLS_CS_VTEC_daily_stats.csv")
OUT_MATCH    = Path("/home/mk-laptop/Downloads/TEC_DATA/2015_2025_MS_WLS_CS_match_rates.csv")

DATE_START = "2015-10-01"
DATE_END   = "2025-09-26"

RX_XYZ = (5411106.3084, -747628.5974, 3286931.3223)

ELEV_MIN_DEG_BASE  = 10.0
ELEV_MIN_DEG_BOOST = 15.0
R_EARTH = 6378137.0
H_IONO  = 350_000.0

f1 = 1575.42e6
f2 = 1227.60e6
K_m_per_TECU = 40.3e16 * (1.0/f2**2 - 1.0/f1**2)  # m/TECU

MERGE_TOL = "2h"
MAX_VTEC_TECU = 200.0
Hampel_k = 3.0
GIM_OFFSET_FLAG_TECU = 20.0

# ---------- GEO / NAV ----------
def ecef_to_geodetic(x, y, z):
    a=6378137.0; f=1/298.257223563; e2=f*(2-f); b=a*(1-f); ep2=(a*a-b*b)/(b*b)
    r=math.hypot(x,y); E2=a*a-b*b; F=54*b*b*z*z; G=r*r+(1-e2)*z*z-e2*E2
    c=(e2*e2*F*r*r)/(G*G*G); s=(1+c+math.sqrt(c*c+2*c))**(1/3)
    P=F/(3*(s+1/s+1)**2*G*G); Q=math.sqrt(1+2*e2*e2*P)
    r0=-(P*e2*r)/(1+Q)+math.sqrt(0.5*a*a*(1+1/Q)-(P*(1-e2)*z*z)/(Q*(1+Q))-0.5*P*r*r)
    U=math.sqrt((r-e2*r0)**2+z*z); V=math.sqrt((r-e2*r0)**2+(1-e2)*z*z)
    Z0=b*b*z/(a*V); h=U*(1-b*b/(a*V)); lat=math.atan2(z+ep2*Z0, r); lon=math.atan2(y,x)
    return lat,lon,h

def elevation_deg(rx_xyz, sat_xyz):
    x,y,z = rx_xyz
    lat,lon,_ = ecef_to_geodetic(x,y,z)
    sl,cl = math.sin(lat), math.cos(lat); slon,clon = math.sin(lon), math.cos(lon)
    R = np.array([[-slon,        clon,       0.0],
                  [-clon*sl, -slon*sl,      cl ],
                  [ clon*cl,  slon*cl,      sl ]])
    d = np.array([sat_xyz[0]-x, sat_xyz[1]-y, sat_xyz[2]-z])
    e,n,u = R @ d
    return math.degrees(math.atan2(u, math.hypot(e,n)))

def mapping_M(el_deg, Re=R_EARTH, H=H_IONO):
    el = np.radians(el_deg)
    sin_zp = (Re/(Re + H)) * np.cos(el)
    sin_zp = np.clip(sin_zp, -1.0, 1.0)
    zp = np.arcsin(sin_zp)
    return 1.0/np.cos(zp)

def sat_ecef_from_navrow(t_utc, rnav):
    MU=3.986005e14; OMEGA_E=7.2921151467e-5
    t = pd.Timestamp(t_utc); t = t.tz_localize("UTC") if t.tzinfo is None else t.tz_convert("UTC")
    toe_time = pd.to_datetime(rnav["ephem_time"], utc=True)
    tk = (t - toe_time).total_seconds(); half=302400.0
    if tk>half: tk-=2*half
    if tk<-half: tk+=2*half
    A=float(rnav["sqrtA"])**2; dn=float(rnav["d_n"]); ecc=float(rnav["e"]); M0=float(rnav["M0"])
    Cuc=float(rnav["Cuc"]); Cus=float(rnav["Cus"]); Crc=float(rnav["Crc"]); Crs=float(rnav["Crs"])
    Cic=float(rnav["Cic"]); Cis=float(rnav["Cis"]); i0=float(rnav["i0"]); IDOT=float(rnav["IDOT"])
    OMG0=float(rnav["Omega0"]); OMG_DOT=float(rnav["OMEGA_DOT"]); omg=float(rnav["omega"]); toe_s=float(rnav["Toe"])
    n0=math.sqrt(MU/A**3); n=n0+dn; M=M0+n*tk
    E=M
    for _ in range(12):
        dE=(M+ecc*math.sin(E)-E)/(1-ecc*math.cos(E)); E+=dE
        if abs(dE)<1e-13: break
    v=math.atan2(math.sqrt(1-ecc*ecc)*math.sin(E), math.cos(E)-ecc)
    phi=v+omg
    du=Cuc*math.cos(2*phi)+Cus*math.sin(2*phi); dr=Crc*math.cos(2*phi)+Crs*math.sin(2*phi); di=Cic*math.cos(2*phi)+Cis*math.sin(2*phi)
    u=phi+du; r=A*(1-ecc*math.cos(E))+dr; i=i0+IDOT*tk+di
    x_orb=r*math.cos(u); y_orb=r*math.sin(u)
    OMG=OMG0+(OMG_DOT-OMEGA_E)*tk - OMEGA_E*toe_s
    cosO,sinO = math.cos(OMG), math.sin(OMG); cosi,sini = math.cos(i), math.sin(i)
    x= x_orb*cosO - y_orb*cosi*sinO
    y= x_orb*sinO + y_orb*cosi*cosO
    z= y_orb*sini
    return x,y,z

# ---------- I/O ----------
def load_nav(path: Path) -> pd.DataFrame:
    nav = pd.read_csv(path)
    nav["prn"] = nav["prn"].str.upper().str.strip()
    nav["toc"] = pd.to_datetime(nav["toc"], utc=True, errors="coerce")
    nav["ephem_time"] = (pd.to_datetime("1980-01-06", utc=True)
                         + pd.to_timedelta(nav["week"]*7, unit="D")
                         + pd.to_timedelta(nav["Toe"], unit="s"))
    keep = ["prn","ephem_time","week","Toe","sqrtA","d_n","e","M0","Cuc","Cus","Crc","Crs","Cic","Cis","i0","IDOT","Omega0","OMEGA_DOT","omega"]
    return nav[keep].sort_values(["prn","ephem_time"]).reset_index(drop=True)

def load_dcb(path: Path) -> pd.DataFrame:
    dcb = pd.read_csv(path)
    dcb["prn"] = dcb["prn"].str.upper().str.strip()
    base = pd.to_datetime(dcb["year"].astype(int).astype(str), format="%Y", utc=True)
    dcb["dcb_date"] = (base + pd.to_timedelta(dcb["doy"].astype(int)-1, unit="D")).dt.floor("D")
    dcb = dcb.rename(columns={"C2W-C1C_sat_m":"sat_dcb_m"})
    return dcb[["prn","dcb_date","sat_dcb_m"]]

def load_drx(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["date"] = pd.to_datetime(df["date"], utc=True).dt.floor("D")
    # robust naming
    if "D_rx_MS_m" not in df.columns:
        c = [c for c in df.columns if c.lower().startswith("d_rx_ms")] or [c for c in df.columns if "MS" in c]
        if c: df = df.rename(columns={c[0]:"D_rx_MS_m"})
    if "D_rx_WLS_m" not in df.columns:
        c = [c for c in df.columns if c.lower().startswith("d_rx_wls")] or [c for c in df.columns if "WLS" in c]
        if c: df = df.rename(columns={c[0]:"D_rx_WLS_m"})
    need = {"date","D_rx_MS_m","D_rx_WLS_m"}
    if not need.issubset(df.columns): raise ValueError("DRX_FILE must have: date, D_rx_MS_m, D_rx_WLS_m")
    return df[list(need)]

# --- obs files ---
OBS_FILES = sorted([*Path(OBS_DIR).glob("*.parquet"), *Path(OBS_DIR).glob("*.csv")])
print(f"📄 Fichiers obs trouvés: {len(OBS_FILES)}")

def _read_day_slice(fp: Path, t0: pd.Timestamp, t1: pd.Timestamp) -> pd.DataFrame:
    cols = ["time","prn","P2_minus_C1_m"]
    if fp.suffix.lower()==".parquet":
        try:
            df = pd.read_parquet(fp, columns=cols, engine="pyarrow",
                                 filters=[("time", ">=", t0), ("time", "<", t1)])
        except Exception:
            df = pd.read_parquet(fp, columns=cols)
            df["time"] = pd.to_datetime(df["time"], utc=True); df = df[(df["time"]>=t0)&(df["time"]<t1)]
    else:
        df = pd.read_csv(fp, usecols=cols, parse_dates=["time"])
        df["time"] = pd.to_datetime(df["time"], utc=True); df = df[(df["time"]>=t0)&(df["time"]<t1)]
    return df

def stack_obs_for_day(t0: pd.Timestamp) -> pd.DataFrame | None:
    t1 = t0 + pd.Timedelta(days=1)
    parts=[]
    for fp in OBS_FILES:
        df = _read_day_slice(fp, t0, t1)
        if df is not None and len(df): parts.append(df)
    if not parts: return None
    obs = pd.concat(parts, ignore_index=True)
    obs["prn"]  = obs["prn"].str.upper().str.strip()
    obs["time"] = pd.to_datetime(obs["time"], utc=True)
    obs = obs.dropna(subset=["time","prn","P2_minus_C1_m"])
    return obs.sort_values(["prn","time"], kind="mergesort").reset_index(drop=True)

def merge_nav(obs_day: pd.DataFrame, nav: pd.DataFrame, tol: str | None) -> pd.DataFrame:
    if obs_day is None or obs_day.empty: return pd.DataFrame()
    out=[]
    for prn_val, left in obs_day.groupby("prn", sort=False):
        right = nav[nav["prn"]==prn_val]
        if right.empty: continue
        left  = left.sort_values("time", kind="mergesort")
        right = right.sort_values("ephem_time", kind="mergesort")
        kw=dict(direction="nearest"); 
        if tol: kw["tolerance"]=pd.Timedelta(tol)
        m = pd.merge_asof(left=left, right=right, left_on="time", right_on="ephem_time", **kw)
        if m is None or m.empty: continue
        if "prn_x" in m.columns and "prn" not in m.columns: m = m.rename(columns={"prn_x":"prn"})
        if "prn_y" in m.columns: m = m.drop(columns=["prn_y"])
        if "prn" not in m.columns: m["prn"] = prn_val
        out.append(m)
    if not out: return pd.DataFrame()
    m = pd.concat(out, ignore_index=True)
    return m.dropna(subset=["ephem_time"]).copy()

# ---------- DCB & mapping ----------
def attach_sat_dcb(m: pd.DataFrame, dcb_day: pd.DataFrame) -> pd.DataFrame:
    m2 = m.merge(dcb_day, on="prn", how="inner").dropna(subset=["sat_dcb_m"])
    if m2.empty: return m2
    m2["y_m"] = m2["P2_minus_C1_m"] - m2["sat_dcb_m"]
    return m2

def mapping_block(df: pd.DataFrame):
    df["M"] = mapping_M(df["elev_deg"].values)
    df["P"] = K_m_per_TECU * df["M"].values
    return df

# ---------- GIM ----------
def load_gim(path: Path) -> pd.DataFrame | None:
    if path is None or not Path(path).exists(): return None
    g = pd.read_csv(path, parse_dates=["time"])
    g["time"] = pd.to_datetime(g["time"], utc=True)
    return g[["time","vtec_gim"]]

def daily_gim_offset(vt_df: pd.DataFrame, gim: pd.DataFrame) -> float | None:
    if gim is None or vt_df is None or vt_df.empty: return None
    g = gim.set_index("time")["vtec_gim"].sort_index()
    v = vt_df.set_index("time")["VTEC_TECU"].sort_index()
    g_al = g.reindex(v.index, method="nearest", tolerance=pd.Timedelta("15min"))
    d = (v - g_al).dropna()
    return float(d.median()) if not d.empty else None

# ---------- VTEC from fixed D_rx ----------
def vtec_from_fixed_drx(df_day: pd.DataFrame, D_rx_m: float, elev_min: float) -> tuple[pd.DataFrame, dict]:
    df = df_day[df_day["elev_deg"] >= elev_min].copy()
    if df.empty: return pd.DataFrame(), {}
    df = mapping_block(df)
    vtec_i = (df["y_m"].values - D_rx_m) / df["P"].values
    df["VTEC_i"] = vtec_i
    df = df[(df["VTEC_i"] >= 0) & (df["VTEC_i"] <= MAX_VTEC_TECU)]
    if df.empty: return pd.DataFrame(), {}
    vt = df.groupby("time", sort=True)["VTEC_i"].median().rename("VTEC_TECU").reset_index()
    diag = {"N_obs_used": int(len(df)),
            "N_epochs":   int(len(vt)),
            "Nsat_mean":  float(df.groupby("time").size().mean()) if len(df) else np.nan}
    return vt, diag

# ---------- 30-min bin (+ GIM) ----------
def bin_30min_with_gim(vt_df: pd.DataFrame, gim: pd.DataFrame | None) -> pd.DataFrame:
    if vt_df is None or vt_df.empty: return pd.DataFrame()
    s = vt_df.set_index("time")["VTEC_TECU"].sort_index()
    r = s.resample("30min")
    v30 = pd.DataFrame({
        "time":        r.median().index,
        "VTEC_median": r.median().values,
        "VTEC_mean":   r.mean().values,
        "VTEC_q25":    r.quantile(0.25).values,
        "VTEC_q75":    r.quantile(0.75).values,
        "N":           r.count().values
    }).dropna(subset=["VTEC_median","N"])
    med = v30["VTEC_median"]
    if med.notna().sum() >= 10:
        m0 = med.median(); mad = (med-m0).abs().median()
        thr = Hampel_k*1.4826*mad if mad>0 else np.inf
        v30.loc[(med-m0).abs()>thr, ["VTEC_median","VTEC_mean"]] = np.nan
    if gim is not None and not gim.empty:
        g = gim.set_index("time")["vtec_gim"].sort_index()
        g_al = g.reindex(v30["time"], method="nearest", tolerance=pd.Timedelta("15min"))
        v30["vtec_gim"] = g_al.to_numpy()
    else:
        v30["vtec_gim"] = np.nan
    return v30.dropna(how="all")

def append_csv(path: Path, df: pd.DataFrame):
    if df is None or df.empty: return
    header = not path.exists()
    df.to_csv(path, mode="a", header=header, index=False)

# ---------- MAIN ----------
def main():
    # load inputs
    nav = load_nav(NAV_FILE)
    dcb = load_dcb(DCB_FILE)
    drx = load_drx(DRX_FILE)
    gim = load_gim(GIM_FILE)

    # days = intersection des jours disponibles en DCB et DRX + fenêtre
    days_dcb = pd.to_datetime(sorted(dcb["dcb_date"].unique()), utc=True)
    days_drx = pd.to_datetime(sorted(drx["date"].unique()), utc=True)
    all_days = pd.Index(days_dcb).intersection(days_drx)
    if DATE_START: all_days = all_days[all_days >= pd.Timestamp(DATE_START, tz="UTC")]
    if DATE_END:   all_days = all_days[all_days <= pd.Timestamp(DATE_END,   tz="UTC")]
    all_days = all_days.sort_values()
    print(f"🗓️  Jours: {len(all_days)} de {all_days[0].date()} à {all_days[-1].date()}")

    # clear outputs
    for p in [OUT_30MIN_MS, OUT_30MIN_WL, OUT_DAILY_MS, OUT_DAILY_WL, OUT_MATCH]:
        if p.exists(): p.unlink()

    match_rows = []

    for t0 in all_days:
        day = str(t0.date())
        # D_rx par technique
        row = drx[drx["date"]==t0.floor("D")]
        if row.empty:
            print(f"⏭️ {day}: D_rx manquant."); continue
        D_ms  = float(row["D_rx_MS_m"].iloc[0])
        D_wls = float(row["D_rx_WLS_m"].iloc[0])

        # obs + nav
        obs_day = stack_obs_for_day(t0)
        if obs_day is None or obs_day.empty:
            print(f"⏭️ {day}: pas d’observations."); continue
        m = merge_nav(obs_day, nav, tol=MERGE_TOL)
        total_obs = int(len(obs_day))
        matched   = 0 if m is None or m.empty else int(m["ephem_time"].notna().sum())
        rate = 100.0*matched/max(total_obs,1)
        if matched>0:
            sel = m["ephem_time"].notna()
            dt_min = (m.loc[sel,"time"] - m.loc[sel,"ephem_time"]).dt.total_seconds()/60.0
            p50 = float(np.nanmedian(np.abs(dt_min))); p95 = float(np.nanpercentile(np.abs(dt_min),95))
        else:
            p50=p95=np.nan
        match_rows.append({"date": t0.floor("D"), "total_obs": total_obs, "matched_obs": matched,
                           "rate_pct": rate, "abs_dt_p50_min": p50, "abs_dt_p95_min": p95})
        if m is None or m.empty:
            print(f"⚠️ {day}: appariement NAV nul."); continue

        # DCB du jour
        dcb_day = dcb[dcb["dcb_date"]==t0.floor("D")][["prn","sat_dcb_m"]]
        if dcb_day.empty:
            print(f"⏭️ {day}: DCB manquant."); continue

        # géométrie
        sv = np.vstack([sat_ecef_from_navrow(row["time"], row) for _,row in m.iterrows()])
        m["sv_x"],m["sv_y"],m["sv_z"] = sv[:,0], sv[:,1], sv[:,2]
        m["elev_deg"] = [elevation_deg(RX_XYZ, (x,y,z)) for x,y,z in zip(m["sv_x"],m["sv_y"],m["sv_z"])]

        # observable corrigée
        m = attach_sat_dcb(m, dcb_day)
        if m.empty:
            print(f"⏭️ {day}: aucun PRN utilisable."); continue

        # ---- MS: VTEC avec D_rx_MS ----
        vt_ms, diag_ms = vtec_from_fixed_drx(m, D_ms, elev_min=ELEV_MIN_DEG_BASE)
        if vt_ms is None or vt_ms.empty:
            print(f"⏭️ {day}: MS solvabilité insuffisante."); 
        else:
            gim_off_ms = daily_gim_offset(vt_ms, gim) if gim is not None else None
            used_mitig_ms = False
            if gim_off_ms is not None and abs(gim_off_ms) >= GIM_OFFSET_FLAG_TECU:
                m2 = m[m["elev_deg"] >= ELEV_MIN_DEG_BOOST].copy()
                vt2, d2 = vtec_from_fixed_drx(m2, D_ms, elev_min=ELEV_MIN_DEG_BOOST)
                if vt2 is not None and not vt2.empty: vt_ms, diag_ms, used_mitig_ms = vt2, d2, True
            # 30 min + GIM
            v30_ms = bin_30min_with_gim(vt_ms, gim)
            if not v30_ms.empty:
                append_csv(OUT_30MIN_MS, v30_ms.assign(date_utc=t0.floor("D")))
            # daily
            vt_idx = vt_ms.set_index("time")["VTEC_TECU"].sort_index()
            gd = vt_idx.agg(VTEC_median="median", VTEC_mean="mean",
                            VTEC_min="min", VTEC_max="max",
                            VTEC_q25=lambda s: s.quantile(0.25),
                            VTEC_q75=lambda s: s.quantile(0.75),
                            VTEC_std="std", N="count").to_frame().T
            gd.insert(0, "date", t0.floor("D"))
            gd["D_rx_m_MS"] = float(D_ms)
            gd["gim_offset_tecu"] = float(gim_off_ms) if gim_off_ms is not None else np.nan
            gd["offset_flag_abs_ge_20"] = bool(gim_off_ms is not None and abs(gim_off_ms) >= GIM_OFFSET_FLAG_TECU)
            gd["mitigation_used"] = bool(used_mitig_ms)
            gd["N_epochs"] = diag_ms.get("N_epochs", np.nan)
            gd["Nsat_mean"] = diag_ms.get("Nsat_mean", np.nan)
            append_csv(OUT_DAILY_MS, gd)

        # ---- WLS: VTEC avec D_rx_WLS (fixe) ----
        vt_wl, diag_wl = vtec_from_fixed_drx(m, D_wls, elev_min=ELEV_MIN_DEG_BASE)
        if vt_wl is None or vt_wl.empty:
            print(f"⏭️ {day}: WLS solvabilité insuffisante."); 
        else:
            gim_off_wl = daily_gim_offset(vt_wl, gim) if gim is not None else None
            used_mitig_wl = False
            if gim_off_wl is not None and abs(gim_off_wl) >= GIM_OFFSET_FLAG_TECU:
                m2 = m[m["elev_deg"] >= ELEV_MIN_DEG_BOOST].copy()
                vt2, d2 = vtec_from_fixed_drx(m2, D_wls, elev_min=ELEV_MIN_DEG_BOOST)
                if vt2 is not None and not vt2.empty: vt_wl, diag_wl, used_mitig_wl = vt2, d2, True
            # 30 min + GIM
            v30_wl = bin_30min_with_gim(vt_wl, gim)
            if not v30_wl.empty:
                append_csv(OUT_30MIN_WL, v30_wl.assign(date_utc=t0.floor("D")))
            # daily
            vt_idx = vt_wl.set_index("time")["VTEC_TECU"].sort_index()
            gd = vt_idx.agg(VTEC_median="median", VTEC_mean="mean",
                            VTEC_min="min", VTEC_max="max",
                            VTEC_q25=lambda s: s.quantile(0.25),
                            VTEC_q75=lambda s: s.quantile(0.75),
                            VTEC_std="std", N="count").to_frame().T
            gd.insert(0, "date", t0.floor("D"))
            gd["D_rx_m_WLS"] = float(D_wls)
            gd["gim_offset_tecu"] = float(gim_off_wl) if gim_off_wl is not None else np.nan
            gd["offset_flag_abs_ge_20"] = bool(gim_off_wl is not None and abs(gim_off_wl) >= GIM_OFFSET_FLAG_TECU)
            gd["mitigation_used"] = bool(used_mitig_wl)
            gd["N_epochs"] = diag_wl.get("N_epochs", np.nan)
            gd["Nsat_mean"] = diag_wl.get("Nsat_mean", np.nan)
            append_csv(OUT_DAILY_WL, gd)

        print(f"✅ {day}  D_rx_MS={D_ms:.3f}  D_rx_WLS={D_wls:.3f}  match={rate:.1f}%")

    pd.DataFrame(match_rows).sort_values("date").to_csv(OUT_MATCH, index=False)
    print(f"\n→ 30min MS : {OUT_30MIN_MS}\n→ 30min WLS : {OUT_30MIN_WL}\n→ Daily MS : {OUT_DAILY_MS}\n→ Daily WLS : {OUT_DAILY_WL}\n→ Match : {OUT_MATCH}")

if __name__ == "__main__":
    pd.set_option("display.width", 160)
    pd.set_option("display.max_columns", 20)
    main()


Generate dataframe with solar and geo indices with daily max

In [ ]:
import re
from pathlib import Path
from typing import Union, List
import numpy as np
from datetime import datetime, timezone, date, timedelta
import pandas as pd

In [ ]:
# ============================
# PIPELINE FUSION (UTC) VTEC + F107 + Kp (GFZ) + Ap + SN + Dst (Kyoto) + heure du max VTEC
# ============================
# --- CONFIG À ADAPTER ---
STATS_CSV = Path(r"C:/Users/mkmoh/Dropbox/1-DATA/TEC_DATA/New_Data/2015_2025_MS_CS_VTEC_daily_stats.csv")
CSV_30MIN = Path(r"C:/Users/mkmoh/Dropbox/1-DATA/TEC_DATA/New_Data/2015_2025_MS_CS_VTEC_30min_stats.csv")             # time (UTC), VTEC_median (au moins)
CSV_F107  = Path(r"C:/Users/mkmoh/Dropbox/1-DATA/TEC_DATA/New_Data/Indices/F10_7_102015-092025.csv")
FILE_KP   = Path(r"C:\Users\mkmoh\Dropbox\1-DATA\TEC_DATA\New_Data\Indices\GFZ_all_indices.txt")
FILE_DST  = Path(r"C:/Users/mkmoh/Dropbox/1-DATA/TEC_DATA/New_Data/Indices/Kyoto_DST_index.txt")

LOCAL_TZ  = "Africa/Casablanca"
# --- Seuils (modifiable si besoin) ---
THR_F107_LOW = 120.0   # sfu
THR_KP_Q     = 2.0     # Kp_max <= 2
THR_AP_Q     = 7.0     # Ap <= 7
THR_DST_Q    = -20.0   # Dst_min >= -20 nT
THR_F107  = 120.0
THR_KP    = 3.0     # max < 4 (par défaut) ; mets agg='mean' côté parseur si tu veux la moyenne
THR_DST   = -30.0   # Dst_min > -50 nT

# ---------- Helpers ----------
def _to_utc_series(tcol: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(tcol):
        return pd.to_datetime((tcol.astype(float) - 2440587.5) * 86400.0, unit="s", utc=True)
    return pd.to_datetime(tcol, utc=True, errors="coerce")

def _coerce_date_utc_ts(df: pd.DataFrame, col: str = "date_utc") -> pd.DataFrame:
    if col not in df.columns:
        return df
    s = pd.to_datetime(df[col], errors="coerce", utc=True)
    s = s.dt.tz_convert("UTC").dt.normalize()
    df[col] = s
    return df

# ---------- F10.7 (CSV) -> jour UTC ----------
def load_f107_utc(csv_path: Union[str, Path]) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    tcol, obs_col, adj_col = "time", "F10_7_obs", "F10_7_adj"
    if tcol not in df.columns:
        raise ValueError("F10.7: colonne 'time' manquante.")
    if obs_col not in df.columns:
        cand = [c for c in df.columns if "obs" in c.lower().replace(" ", "") and any(k in c.lower() for k in ("10.7","f107","f10_7"))]
        if not cand:
            raise ValueError("F10.7: colonne 'observed' introuvable.")
        obs_col = cand[0]
    ts_utc = _to_utc_series(df[tcol])
    out = pd.DataFrame({
        "date_utc": ts_utc.dt.floor("D"),
        "f107_obs": pd.to_numeric(df[obs_col], errors="coerce"),
        "f107_adj": pd.to_numeric(df[adj_col], errors="coerce") if adj_col in df.columns else np.nan
    }).dropna(subset=["date_utc"])
    out = out.drop_duplicates(subset=["date_utc"], keep="last").sort_values("date_utc")
    return _coerce_date_utc_ts(out, "date_utc")

# ---------- GFZ (texte) -> Kp journaliers + Ap + SN en UTC ----------
# Format GFZ: 40 lignes header '#', puis 1 ligne/jour, tokens:
# 0:Y 1:M 2:D 3:days 4:days_m 5:BSR 6:dB 7..14: 8 Kp  15..22: 8 ap  23:Ap  24:SN  25:Fobs  26:Fadj  27:D
def parse_gfz_kp_daily_utc(path: Path) -> pd.DataFrame:
    if not path or not path.exists():
        return pd.DataFrame(columns=["date_utc"])
    rows = []
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if not line.strip() or line.startswith("#"):
                continue
            toks = line.split()
            if len(toks) < 27:
                # fallback minimal si ligne atypique
                try:
                    year = int(line[0:4]); month = int(line[5:7]); day = int(line[8:10])
                except Exception:
                    continue
                dt = pd.Timestamp(datetime(year, month, day, tzinfo=timezone.utc))
                rows.append({"date_utc": dt, "kp_daily_mean": np.nan, "kp_daily_max": np.nan, "Ap": np.nan, "SN": np.nan})
                continue
            try:
                year, month, day = map(int, toks[:3])
                kp_vals = np.array(list(map(float, toks[7:15])), float)     # 8×Kp
                ap_vals = np.array(list(map(float, toks[15:23])), float)    # 8×ap (pas utilisé ici)
                Ap = float(toks[23]) if toks[23] not in ("-1","9999") else np.nan
                SN = float(toks[24]) if toks[24] not in ("-1","9999") else np.nan
            except Exception:
                continue

            dt = pd.Timestamp(datetime(year, month, day, tzinfo=timezone.utc))
            rows.append({
                "date_utc":      dt,
                "kp_daily_mean": float(np.nanmean(kp_vals)) if np.isfinite(np.nanmean(kp_vals)) else np.nan,
                "kp_daily_max":  float(np.nanmax(kp_vals))  if np.isfinite(np.nanmax(kp_vals))  else np.nan,
                "Ap":            Ap,
                "SN":            SN,
            })
    df = pd.DataFrame(rows).sort_values("date_utc").reset_index(drop=True)
    return _coerce_date_utc_ts(df, "date_utc")

# ---------- Dst Kyoto (texte) -> stats journalières UTC ----------
def parse_dst_kyoto_daily_utc(path: Path) -> pd.DataFrame:
    if not path or not path.exists():
        return pd.DataFrame(columns=["date_utc"])
    pat = re.compile(r"^DST(?P<yy>\d{2})(?P<mm>\d{2})\*(?P<dd>\d{2})")
    rows = []
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            m = pat.match(line)
            if not m:
                continue
            yy = int(m.group("yy")); mm = int(m.group("mm")); dd = int(m.group("dd"))
            year = 2000 + yy  # période 2015–2021 → on force 20xx
            # extraire nombres après le tag X###
            after = re.split(r"X\d{3}", line, maxsplit=1)
            vals = []
            if len(after) == 2:
                nums = re.findall(r"[-]?\d+", after[1])
                vals = [float(n) for n in nums]
            if len(vals) < 24:
                continue
            hourly = np.array(vals[-24:], dtype=float)  # 24 dernières valeurs = heures 0..23
            dt = pd.Timestamp(datetime(year, mm, dd, tzinfo=timezone.utc))
            rows.append({
                "date_utc":  dt,
                "Dst_min":   float(np.nanmin(hourly)) if np.isfinite(np.nanmin(hourly)) else np.nan,
                "Dst_mean":  float(np.nanmean(hourly)) if np.isfinite(np.nanmean(hourly)) else np.nan,
                "Dst_n":     int(np.sum(~np.isnan(hourly))),
            })
    df = pd.DataFrame(rows).sort_values("date_utc").reset_index(drop=True)
    return _coerce_date_utc_ts(df, "date_utc")

# ---------- Max quotidien depuis la série 30 min (UTC) ----------
def daily_max_from_30min_utc(csv_30min: Union[str, Path],
                             time_col="time",
                             tec_col="VTEC_median") -> pd.DataFrame:
    raw = pd.read_csv(csv_30min)
    ts_utc = pd.to_datetime(raw[time_col], errors="coerce", utc=True)
    df = pd.DataFrame({
        "date_utc": ts_utc.dt.floor("D"),
        "slot_utc": (ts_utc.dt.hour*2 + (ts_utc.dt.minute//30)).astype("Int64"),
        "ts_utc":   ts_utc,
        "tec":      pd.to_numeric(raw[tec_col], errors="coerce")
    }).dropna(subset=["tec"])
    df = df.sort_values(["date_utc","slot_utc","ts_utc"]).drop_duplicates(["date_utc","slot_utc"], keep="last")
    if df.empty:
        return pd.DataFrame(columns=["date_utc","max_bin_utc","max_ts_utc","VTEC_max_from_30min","max_hour_utc","max_time_utc_str"])
    idx = df.groupby("date_utc")["tec"].idxmax()
    out = df.loc[idx, ["date_utc","slot_utc","ts_utc","tec"]].rename(
        columns={"slot_utc":"max_bin_utc","ts_utc":"max_ts_utc","tec":"VTEC_max_from_30min"}
    ).sort_values("date_utc")
    out["max_hour_utc"] = out["max_ts_utc"].dt.hour + out["max_ts_utc"].dt.minute/60.0
    out["max_time_utc_str"] = out["max_ts_utc"].dt.strftime("%H:%M")
    return out

# ============================
#        EXÉCUTION
# ============================

# 1) Stats VTEC quotidiennes (UTC)
stats = pd.read_csv(STATS_CSV, parse_dates=["date"])
if stats["date"].dt.tz is None:
    stats["date_utc"] = stats["date"].dt.tz_localize("UTC").dt.normalize()
else:
    stats["date_utc"] = stats["date"].dt.tz_convert("UTC").dt.normalize()
stats = _coerce_date_utc_ts(stats, "date_utc")

# 2) Indices (UTC)
f107 = load_f107_utc(CSV_F107)             # date_utc, f107_obs, f107_adj
kp   = parse_gfz_kp_daily_utc(FILE_KP)     # date_utc, kp_daily_mean, kp_daily_max, Ap, SN
dst  = parse_dst_kyoto_daily_utc(FILE_DST) # date_utc, Dst_min, Dst_mean, Dst_n

# 3) Heure du max VTEC (UTC)
dmax = daily_max_from_30min_utc(CSV_30MIN)

# Harmoniser la clé
for _df in (stats, f107, kp, dst, dmax):
    _coerce_date_utc_ts(_df, "date_utc")

# 4) Fusion
daily = (
    stats
    .merge(dmax, on="date_utc", how="left")
    .merge(f107, on="date_utc", how="left")
    .merge(kp,   on="date_utc", how="left")
    .merge(dst,  on="date_utc", how="left")
    
)

# 5) Ordonner les colonnes (si présentes)
first = [
    "date_utc", "VTEC_median","VTEC_mean","VTEC_min","VTEC_max","VTEC_q25","VTEC_q75","VTEC_std","N","D_rx_m","VTEC_max_from_30min", 
    "max_bin_utc","max_time_utc_str","max_hour_utc","max_ts_utc","gim_offset_tecu","offset_flag_abs_ge_20","mitigation_used", "f107_obs","f107_adj", "kp_daily_mean","kp_daily_max","Ap","SN", "Dst_min",
    "Dst_mean","Dst_n"]
cols = [c for c in first if c in daily.columns] + [c for c in daily.columns if c not in first]
daily = daily[cols]



# 1) Label solaire (Low / High / NA)
daily["solar_label"] = np.where(daily["f107_obs"].notna(), np.where(daily["f107_obs"] < THR_F107_LOW, "Low", "High"),"NA")

# 2) Label géomagnétique (Q / NQ / NA)
#have_all = daily[["kp_daily_max", "Ap", "Dst_min"]].notna().all(axis=1)
#quiet_mask = (
#    (daily["kp_daily_max"] <= THR_KP_Q) &
#    (daily["Ap"]          <= THR_AP_Q) &
#    (daily["Dst_min"]     >= THR_DST_Q)
#)

#daily["geomag_label"] = np.where(
#    ~have_all, "NA",
#    np.where(quiet_mask, "Q", "NQ")
#)


# 6) Sauvegarde
OUT = STATS_CSV.with_name(STATS_CSV.stem + "_UTC_with_indices_and_max.csv")
daily.to_csv(OUT, index=False)

# 7) Quick log
print(f"✅ Fusion UTC sauvegardée: {OUT}")
for col in ["f107_obs","kp_daily_mean","kp_daily_max","Ap","SN","Dst_min","VTEC_max_from_30min"]:
    if col in daily.columns:
        print(f"Manquants — {col}: {daily[col].isna().sum()}")

#print(daily.head(3))

daily.head(5)

Générer une colonne indiquant le label Q ou D à partir des données GFZ